# PRML Final PJ Part 1
## Adversarial Training for Chinese SSC(Single Sentence Classification)

### 初始化代码

In [ ]:
%reload_ext autoreload
%autoreload 2
import initializer
from initializer import *


### 正常训练

In [ ]:
# 不使用FGM

from torch.optim import AdamW,Adam
from runner import Runner
model = Model()
model.to(device)
optimizer = AdamW(model.parameters(), lr=args['init_lr']) # 这个真的是应该小啊！2e-5 1e-5这种才可以，5e-4直接崩盘！50%都不到
criterion = torch.nn.CrossEntropyLoss()
runner = Runner(model,optimizer,loss_fn=criterion)
runner.train(train_loader=loader,valid_loader=loader_valid,num_epoch=args['epoch'])

In [ ]:
runner.predict(test_loader=loader_test)

### 使用FGM

In [ ]:
from transformers import AdamW
from FGM import FGM
from runner import Runner_FGM
model = Model()
model.to(device)
fgm = FGM(model=model)
optimizer = AdamW(model.parameters(), lr=args['init_lr']) 
criterion = torch.nn.CrossEntropyLoss()
runner = Runner_FGM(model,optimizer,loss_fn=criterion,fgm = fgm)
runner.train(train_loader=loader,valid_loader=loader_valid,num_epoch=args['epoch'])

In [ ]:
runner.predict(test_loader=loader_test)


### 使用PGD

In [ ]:
from transformers import AdamW
from PGD import PGD
from runner import Runner_PGD
model = Model()
model.to(device)
pgd = PGD(model=model,k=3,epsilon=1,alpha=0.3)
optimizer = AdamW(model.parameters(), lr=args['init_lr']) 
criterion = torch.nn.CrossEntropyLoss()
runner = Runner_PGD(model,optimizer,loss_fn=criterion,pgd=pgd)
runner.train(train_loader=loader,valid_loader=loader_valid,num_epoch=args['epoch'])

In [ ]:
runner.predict(test_loader=loader_test)

### 使用FreeLB

In [13]:
from transformers import AdamW
from FreeLB import FreeLB
from runner import Runner_FreeLB
model = Model()
model.to(device)
freelb = FreeLB(model=model,k=3,epsilon=0.2,alpha=5e-2)
optimizer = AdamW(model.parameters(), lr=args['init_lr']) 
criterion = torch.nn.CrossEntropyLoss()
runner = Runner_FreeLB(model,optimizer,loss_fn=criterion,freelb=freelb)
runner.train(train_loader=loader,valid_loader=loader_valid,num_epoch=args['epoch'])

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Best performance on valid set upgraded: accuracy: 0.5408333333333334
Best performance on valid set upgraded: accuracy: 0.5808333333333333
Best performance on valid set upgraded: accuracy: 0.6041666666666666
Best performance on valid set upgraded: accuracy: 0.7158333333333333
Best performance on valid set upgraded: accuracy: 0.7791666666666667
Best performance on valid set upgraded: accuracy: 0.8058333333333333
Best performance on valid set upgraded: accuracy: 0.845
Best performance on valid set upgraded: accuracy: 0.8516666666666667
[epoch]:1,[step]:10,[loss]:0.35723423957824707,[score]:0.859375
Best performance on valid set upgraded: accuracy: 0.87
Best performance on valid set upgraded: accuracy: 0.8708333333333333
Best performance on valid set upgraded: accuracy: 0.8791666666666667
Best performance on valid set upgraded: accuracy: 0.89
Best performance on valid set upgraded: accuracy: 0.8916666666666667
Best performance on valid set upgraded: accuracy: 0.8958333333333334
[epoch]:1,[

In [14]:
runner.predict(test_loader=loader_test)

Score on test set:0.9483333333333334


0.9483333333333334

### kill ipython内核，清除缓存

In [15]:
import os
 
pid = os.getpid()
!kill -9 $pid

# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# ebd_name = 'word_embeddings'
# for name,params in model.named_parameters():
#     if ebd_name in name:
#         print(name)
# >> pretrained_bert.embeddings.word_embeddings.weight
# 因此只需要 word_embeddings

: 

: 

## 记录


### 12.4
(1,0.3,3)PGD效果奇差无比，模型除了第一轮epoch在验证集上有提升75%左右，后面基本不收敛，训练集准确率维持在50%
(0.1,0.3,1) 仍然不好，在第30次迭代后验证集准确率达到90%，之后就在训练集的准确率就开始下降到50%
又试了(1,1,1) 还不好。然后发现有个地方没写上data